https://github.com/xSzpo/lending-club/blob/master/02_notebooks/04_One_dim_analysis_feature_prep.ipynb      
https://github.com/xSzpo/model_and_abtest/blob/master/notebooks/3.%20model.ipynb    
https://geocoder.readthedocs.io/providers/OpenStreetMap.html    
https://github.com/mediagis/nominatim-docker/tree/master/3.6   


In [17]:
import numpy as np
import pandas as pd
import re
import geocoder


In [19]:
%reload_kedro

In [20]:
df_cust = catalog.load("customers")

In [21]:
d_sex = {1: 'male', 2:'female', 0: 'other'}
df_cust.sex = df_cust.sex.map(lambda x: d_sex[x])

In [22]:
df_cust.shape

(500000, 6)

In [23]:
df_cust.describe()

,customerID,age,Income
count,500000.000000,500000.000000,490025.000000
mean,250000.500000,44.630508,50000.039182
std,144337.711635,16.178283,28899.119696
min,1.000000,5.000000,0.000000
25%,125000.750000,27.000000,25000.000000
50%,250000.500000,50.000000,50000.000000
75%,375000.250000,58.000000,75000.000000
max,500000.000000,90.000000,100000.000000


```
https://github.com/mediagis/nominatim-docker/tree/master/3.6


docker run -it --rm \
  -e PBF_URL=https://download.geofabrik.de/europe/monaco-latest.osm.pbf \
  -e REPLICATION_URL=https://download.geofabrik.de/europe/monaco-updates/ \
  -e IMPORT_WIKIPEDIA=true \
  -p 8080:8080 \
  --name nominatim \
  mediagis/nominatim:3.6
```

In [24]:
residentialAddress = df_cust.residentialAddress.apply(lambda x: re.sub(r'[0-9]+', '', x)).value_counts().index

In [28]:
residentialAddress = df_cust.residentialAddress.sample(10, random_state=1)

In [29]:
residentialAddress_geocoded = []

In [30]:
len(residentialAddress)

10

In [31]:
%%capture

geocoder.osm('warsaw'.strip(), url='http://localhost:8080').json

In [32]:
for adr in residentialAddress:
    residentialAddress_geocoded += [geocoder.osm(adr.strip()).json]


In [33]:
pd.DataFrame(residentialAddress_geocoded).country.value_counts()

Australia                 7
United States             2
New Zealand / Aotearoa    1
Name: country, dtype: int64

In [34]:
df_cust.postalAddress.fillna('x').apply(lambda x: re.sub(r'[0-9]+', '', x)).value_counts()

x                        10119
 Boundary Road             480
 Thomas Street             427
 Victoria Street           390
 Princess Street           357
                         ...  
 Spano Street               12
 Peter doherty Street       12
 Bay Terrace                12
 Nicoro Place               11
 Knapp Street               11
Name: postalAddress, Length: 13261, dtype: int64

In [35]:
df_cust.head()

,customerID,sex,age,residentialAddress,postalAddress,Income
0,1,male,25,28 Irvine Place,28 Irvine Place,<NA>
1,2,male,19,72 Bertha Street,72 Bertha Street,43200
2,3,female,22,63 Ladberry Street,63 Ladberry Street,70200
3,4,other,24,98 Linneman Close,98 Linneman Close,93900
4,5,male,53,56 Salonica Road,56 Salonica Road,77000


In [36]:
df_trans= catalog.load("transactions")

In [37]:
df_trans.shape

(2000000, 8)

In [38]:
d_paymentStatus = {0: 'ok', 1:'late', 2: 'never'}
df_trans.paymentStatus1 = df_trans.paymentStatus1.map(lambda x: d_paymentStatus[x])
df_trans.paymentStatus2 = df_trans.paymentStatus2.map(lambda x: d_paymentStatus[x])
df_trans.paymentStatus3 = df_trans.paymentStatus3.map(lambda x: d_paymentStatus[x])
df_trans.paymentStatus4 = df_trans.paymentStatus4.map(lambda x: d_paymentStatus[x])

In [39]:
df_trans.head()

,transactionID,shopID,customerID,price,paymentStatus1,paymentStatus2,paymentStatus3,paymentStatus4
0,1,114,2869,140,ok,ok,ok,ok
1,2,855,124077,70,ok,ok,ok,ok
2,3,752,86184,85,ok,ok,ok,ok
3,4,246,248861,43,late,ok,ok,ok
4,5,973,162112,127,ok,ok,ok,ok


In [40]:
number_of_weeks = 52

dates = dict([i for i in zip(range(number_of_weeks), pd.date_range(start='2020-01-01', end='2020-12-31',periods=number_of_weeks).map(lambda x: x.date()))])
div = round(df_trans.shape[0]/(number_of_weeks-1),0)
df_trans['pseudo_dates'] = df_trans.transactionID.apply(lambda x: int(round(x/div,0))).map(lambda x: dates[x])

In [41]:
df_trans.customerID.value_counts()

61875     16
270525    16
47943     15
86643     15
110021    15
          ..
329694     1
463876     1
490432     1
440232     1
392512     1
Name: customerID, Length: 490533, dtype: Int64

In [42]:
df_trans.shopID.value_counts()

847    2379
589    2354
280    2354
102    2343
234    2343
       ... 
843    2103
729    2102
276    2093
419    2088
248    2087
Name: shopID, Length: 900, dtype: Int64